# 8-Genre Non-Deep Models
### **Genre Label: 8**
### **Data: Under sampling (max 7000 for train) || clean_df**
### **Naive Bayes, Logistic Regression**

In [1]:
#@title Installs
!pip install pydot --quiet
!pip install gensim --quiet
!pip install tensorflow==2.15.0 --quiet #15 13
!pip install tf_keras==2.15.0 --quiet
!pip install tensorflow-datasets==4.8 --quiet #8
!pip install tensorflow-text==2.15.0 --quiet #15
!pip install transformers==4.17 --quiet #4.40.2 #4.37.2



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.2/475.2 MB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 53.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 83.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 22.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.8 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorstore 0.1.63 requires ml-dtypes>=0.3.1, but you have ml-dtypes 0.2.0 which is incompatible.
tf-keras 2.17.0 requires tensorflow<2.18,>=2.17, but you have tensorflow 2.15.0 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.2/5.2 MB 35.3 MB/s eta 0:00:00


In [18]:
import pandas as pd
import numpy as np
import os
import time
import seaborn as sns
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import StandardScaler
from collections import Counter
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB



from tqdm import tqdm
from multiprocessing import Pool, cpu_count


In [10]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [11]:
cleaned_df = pd.read_pickle('/content/drive/MyDrive/w266/266 Final Project/Music4all Dataset/cleaned_df_8_genre.pkl')

In [12]:
# creating a table to display genre distribution for the entire dataset
genre_counts = cleaned_df['broad_genre'].value_counts().reset_index()
genre_counts.columns = ['genre', 'count']

from tabulate import tabulate
print(tabulate(genre_counts, headers='keys', tablefmt='psql'))

+----+------------------+---------+
|    | genre            |   count |
|----+------------------+---------|
|  0 | Rock             |   15998 |
|  1 | Pop              |   15047 |
|  2 | Jazz and R&B     |   11216 |
|  3 | Country and Folk |   10364 |
|  4 | Punk and Metal   |   10059 |
|  5 | Electronic       |    5406 |
|  6 | Hip Hop & Rap    |    3371 |
|  7 | Reggae           |    2609 |
+----+------------------+---------+


In [15]:
train_df, remaining_df = train_test_split(cleaned_df, test_size=0.20, random_state=42)
val_df, test_df = train_test_split(remaining_df, test_size=0.50, random_state=42)

print('Training set shape:', train_df.shape)
print('Validation set shape:', val_df.shape)
print('Test set shape:', test_df.shape)

Training set shape: (59256, 8)
Validation set shape: (7407, 8)
Test set shape: (7407, 8)


In [16]:
# random undersample to fit the lowest class which is alternative
from sklearn.utils import resample

max_samples = 7000
undersampled_train_df = pd.DataFrame()

for genre in train_df['broad_genre'].unique():
    genre_df = train_df[train_df['broad_genre'] == genre]
    if len(genre_df) > max_samples:
        genre_df = resample(genre_df, replace=False, n_samples=max_samples, random_state=42)
    undersampled_train_df = pd.concat([undersampled_train_df, genre_df])

print('Training set genre distribution:')
print(undersampled_train_df['broad_genre'].value_counts())
train_df = undersampled_train_df

Training set genre distribution:
broad_genre
Pop                 7000
Jazz and R&B        7000
Rock                7000
Country and Folk    7000
Punk and Metal      7000
Electronic          4324
Hip Hop & Rap       2693
Reggae              2060
Name: count, dtype: int64


In [17]:
train_genre_counts = undersampled_train_df['broad_genre'].value_counts().reset_index()
train_genre_counts.columns = ['genre', 'count']


print(tabulate(train_genre_counts, headers='keys', tablefmt='psql'))

+----+------------------+---------+
|    | genre            |   count |
|----+------------------+---------|
|  0 | Pop              |    7000 |
|  1 | Jazz and R&B     |    7000 |
|  2 | Rock             |    7000 |
|  3 | Country and Folk |    7000 |
|  4 | Punk and Metal   |    7000 |
|  5 | Electronic       |    4324 |
|  6 | Hip Hop & Rap    |    2693 |
|  7 | Reggae           |    2060 |
+----+------------------+---------+


In [19]:
# getting the text and tables from train, validation and test
train_text = train_df['lyrics'].tolist()
val_text = val_df['lyrics'].tolist()
test_text = test_df['lyrics'].tolist()

train_label = train_df['broad_genre'].tolist()
val_label = val_df['broad_genre'].tolist()
test_label = test_df['broad_genre'].tolist()

In [21]:
# getting rid of \n in the lyrics column
def preprocess_text(text):
  return text.replace('\n', ' ')

train_text = [preprocess_text(text) for text in train_text]
val_text = [preprocess_text(text) for text in val_text]
test_text = [preprocess_text(text) for text in test_text]


In [22]:
encode_label = LabelEncoder()
train_label = encode_label.fit_transform(train_label)
val_label = encode_label.transform(val_label)
test_label = encode_label.transform(test_label)

In [24]:
# initializeing TF-IDF vectorizer
tfidf_vectorizer = TfidfVectorizer(max_features= 10000)

# fit and transform the text
X_train = tfidf_vectorizer.fit_transform(train_text)
X_val = tfidf_vectorizer.transform(val_text)
X_test = tfidf_vectorizer.transform(test_text)


## Naive Bayes Baseline

In [27]:
# Train the model
model = MultinomialNB()
model.fit(X_train, train_label)

# Predict and evaluate
val_pred = model.predict(X_val)
print('Accuracy:', accuracy_score(val_label, val_pred))
print('Classification Report:\n', classification_report(val_label, val_pred, target_names = encode_label.classes_))

Accuracy: 0.40353719454570003
Classification Report:
                   precision    recall  f1-score   support

Country and Folk       0.45      0.15      0.23      1002
      Electronic       0.33      0.01      0.03       509
   Hip Hop & Rap       0.71      0.51      0.60       343
    Jazz and R&B       0.36      0.63      0.46      1155
             Pop       0.37      0.27      0.31      1489
  Punk and Metal       0.45      0.73      0.56       989
          Reggae       0.54      0.31      0.40       290
            Rock       0.36      0.44      0.40      1630

        accuracy                           0.40      7407
       macro avg       0.45      0.38      0.37      7407
    weighted avg       0.41      0.40      0.37      7407



## Logistic Regression TF-IDF

In [37]:
# initalizing logistic regression for baseline model
logreg_model = LogisticRegression(max_iter=5000, solver='liblinear')

# fitting the model
logreg_model.fit(X_train, train_label)

# evalutating the baseline model for test set
train_predict = logreg_model.predict(X_train)
train_accuracy = accuracy_score(train_label, train_predict)
print(f'Train Accuracy: {train_accuracy}')

#evaluation the baseline model for validation set
val_predict = logreg_model.predict(X_val)
val_accuracy = accuracy_score(val_label, val_predict)
print(f'Validation Accuracy:{val_accuracy}')

Train Accuracy: 0.556389953944234
Validation Accuracy:0.4314837316052383


#Testing

## Naive Bayes

In [28]:
# Predict and evaluate
test_pred = model.predict(X_test)
print('Accuracy:', accuracy_score(test_label, test_pred))
print('Classification Report:\n', classification_report(test_label, test_pred, target_names = encode_label.classes_))

Accuracy: 0.40583232077764275
Classification Report:
                   precision    recall  f1-score   support

Country and Folk       0.47      0.16      0.24      1025
      Electronic       0.56      0.02      0.03       573
   Hip Hop & Rap       0.74      0.55      0.63       335
    Jazz and R&B       0.34      0.62      0.44      1087
             Pop       0.42      0.28      0.34      1536
  Punk and Metal       0.46      0.73      0.56      1051
          Reggae       0.51      0.28      0.36       259
            Rock       0.35      0.45      0.40      1541

        accuracy                           0.41      7407
       macro avg       0.48      0.39      0.38      7407
    weighted avg       0.44      0.41      0.37      7407



## Logistic Regression

In [38]:
#evaluation the baseline model for test set
test_pred = logreg_model.predict(X_test)
print('Accuracy:', accuracy_score(test_label, test_pred))
print('Classification Report:\n', classification_report(test_label, test_pred, target_names = encode_label.classes_))

Accuracy: 0.43391386526258946
Classification Report:
                   precision    recall  f1-score   support

Country and Folk       0.41      0.46      0.43      1025
      Electronic       0.25      0.15      0.19       573
   Hip Hop & Rap       0.67      0.73      0.70       335
    Jazz and R&B       0.39      0.53      0.45      1087
             Pop       0.44      0.34      0.38      1536
  Punk and Metal       0.47      0.71      0.57      1051
          Reggae       0.53      0.32      0.40       259
            Rock       0.43      0.32      0.36      1541

        accuracy                           0.43      7407
       macro avg       0.45      0.44      0.44      7407
    weighted avg       0.43      0.43      0.42      7407

